## Copyright (C) 2020 Sobhan Moradiyan Daghigh - All Rights Reserved
#### > 12/24/2020
### 
## ID3

In [50]:
import numpy as np
import pandas as pd

eps = np.finfo(float).eps

### Reading datasets 

In [56]:
train = pd.DataFrame(pd.read_csv('./datasets/train.csv'))
train_labels = pd.DataFrame(pd.read_csv('./datasets/train_labels.csv'))

In [31]:
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,484,3,"Turkula, Mrs. (Hedwig)",female,63.0,0,0,4134.0,9.5875,NaN,S
1,276,1,"Andrews, Miss. Kornelia Theodosia",female,63.0,1,0,13502.0,77.9583,D7,S
2,223,3,"Green, Mr. George Henry",male,51.0,0,0,21440.0,8.0500,NaN,S
3,632,3,"Lundahl, Mr. Johan Svensson",male,51.0,0,0,347743.0,7.0542,NaN,S
4,26,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077.0,31.3875,NaN,S


In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  800 non-null    int64  
 1   Pclass       800 non-null    int64  
 2   Name         800 non-null    object 
 3   Sex          800 non-null    object 
 4   Age          640 non-null    float64
 5   SibSp        800 non-null    int64  
 6   Parch        800 non-null    int64  
 7   Ticket       589 non-null    float64
 8   Fare         800 non-null    float64
 9   Cabin        183 non-null    object 
 10  Embarked     800 non-null    object 
dtypes: float64(3), int64(4), object(4)
memory usage: 68.9+ KB


#### Therefore, it seems that (Age, Ticket, Cabin) columns have missing values.

### Attrs which wanna use.

In [81]:
attrs = ['Pclass', 'Sex', 'Embarked']

###  Summary of train_labels

In [33]:
train_labels.value_counts()

Survived
0           492
1           308
dtype: int64

### Ok let's define a function for calculating Entropy

In [75]:
def get_entropy(dataset, labels, attr):
    entropy = 0
    
    labels   = labels['Survived'].unique()         # 0: dead, 1: alived
    features = dataset[attr].unique()              # Each feature for an attr. e.g. Sex attr has (Female, Male) features. 
    
    for feature in features:
        feature_entropy = 0
        
        for label in labels:
            cnt_of_each_grp = len(dataset[attr][dataset[attr]==feature][labels['Survived']==label])
            cnt_of_all = len(dataset[attr][dataset[attr]==feature])
            fraction = np.divide(cnt_of_each_grp, np.add(cnt_of_all, eps))
            feature_entropy += np.multiply(-fraction, np.log2(fraction))               
        
        fraction_of_features = np.divide(cnt_of_all, np.add(len(dataset), eps))
        entropy += np.multiply(fraction_of_features, feature_entropy)
        return entropy

### And another one for calculating root entropy 

In [74]:
def get_root_entropy(labels):
    entropy = 0
    
    labels   = labels['Survived'].unique()         # 0: dead, 1: alived
    
    for label in labels:
        cnt_of_each_grp = labels['Survived'].value_counts()[label]
        cnt_of_all = len(labels)
        fraction = np.divide(cnt_of_each_grp, np.add(cnt_of_all, eps))
        entropy += np.multiply(-fraction, np.log2(fraction))               
    
    return entropy

### And finaly a function for calculating Information Gain (IG)

In [72]:
def get_information_gain(dataset, labels, attr):
    root_entropy = get_root_entropy(labels)
    attr_entropy = get_entropy(dataset, labels, attr)
    
    ig = root_entropy - attr_entropy
    
    return ig

In [71]:
get_root_entropy(train_labels)

0.9614969508235551

In [78]:
entropy = {k:get_entropy(train, train_labels, k) for k in ['Pclass', 'Sex', 'Embarked']}
print("Entropy of each atrr: ")
entropy

Entropy of each atrr: 


{'Pclass': 0.4432098091637809,
 'Sex': 0.28643773973420256,
 'Embarked': 0.6734013070701766}

In [256]:
ig = {k:get_information_gain(train, train_labels, k) for k in attrs}
print("IG of each atrr: ")
ig

IG of each atrr: 


{'Pclass': 0.5182871416597743,
 'Sex': 0.6750592110893525,
 'Embarked': 0.2880956437533785}

#### Therefore, till now 'Sex' attr is the best one

### Define a func which choose the most information gain for each iterate.

In [98]:
def get_best_ig(ig):
    best = max(ig.values())
    attr_index = list(ig.values()).index(best)
    attr_name  = list(ig.keys())[attr_index]
    return attr_name

In [99]:
get_best_ig(ig)

'Sex'

### Define a func which returns the subtree for each node.

In [132]:
def get_subtree_df(dataset, attr, feature):
    return pd.DataFrame(dataset[dataset[attr] == feature].reset_index(drop=True))

### Defind a func for checking purity of nodes.

In [248]:
def check_purity(dataset, labels, subtree):
    alived = 0
    dead = 0
    
    for passengerId in subtree.iloc[:, 0]:
        index = dataset.loc[dataset['PassengerId'] == passengerId].index.values[0]
        if labels['Survived'][index] == 0:
            dead += 1
        elif labels['Survived'][index] == 1:
            alived += 1

    if np.divide(max(alived, dead), np.add(alived, dead)) > 0.95:
        return 1, [1 if alived > dead else 0]    # Pure
    else:
        return 0, [1 if alived > dead else 0]    # Not pure

### And finally the main function for making the tree.

In [257]:
def iterate_tree(dataset, labels, attrs, ig_list, tree=None):

    node = get_best_ig(ig_list)
    ig_list.pop(node, None)
    
    features = dataset[node].unique()
    
    # Create a dictionary for our tree, just onece
    if tree is None:           
        tree = {}
        tree[node] = {}
    
    for feature in features:
        subtree = get_subtree_df(dataset, node, feature)
        purity, won_class = check_purity(dataset, labels, subtree)
            
        if purity:
            tree[node][feature] = won_class                                                    
        else:
            if not ig_list:
                tree[node][feature] = won_class
                return tree
            else:
                tree[node][feature] = iterate_tree(dataset, labels, attrs, ig_list)
            
    return tree

In [258]:
subtree = iterate_tree(train, train_labels, attrs, ig)
subtree

{'Sex': {'female': {'Pclass': {3: {'Embarked': {'S': [0]}}, 1: [1]}},
  'male': [0]}}

In [259]:
import pprint
pprint.pprint(subtree)

{'Sex': {'female': {'Pclass': {1: [1], 3: {'Embarked': {'S': [0]}}}},
         'male': [0]}}
